In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

n = 100000

# 2025-ös dátumtartomány
dates = pd.date_range(start="2025-01-01", end="2025-12-31")

# Véletlen kezdés dátumok
kezdes_datum = np.random.choice(dates, size=n)

# Véletlen napeltolás (1–50 nap)
eltolas = np.random.randint(1, 50, size=n)

# Vég dátum = kezdés + eltolás
veg_datum = kezdes_datum + pd.to_timedelta(eltolas, unit="D")

# Véletlen kategóriák
kategoriak = np.random.choice(["kat1", "kat2", "kat3"], size=n)

# DataFrame létrehozása
df = pd.DataFrame({
    "sorszam": range(1, n + 1),
    "kezdes_datum": kezdes_datum,
    "veg_datum": veg_datum,
    "kategoria": kategoriak
})

# Számított oszlop: különbség napokban
df["napok_szama"] = (df["veg_datum"] - df["kezdes_datum"]).dt.days + 1

print(df.head())


   sorszam kezdes_datum  veg_datum kategoria  napok_szama
0        1   2025-06-11 2025-07-08      kat3           28
1        2   2025-12-27 2025-12-28      kat2            2
2        3   2025-11-30 2025-12-05      kat3            6
3        4   2025-08-16 2025-09-29      kat1           45
4        5   2025-07-31 2025-09-06      kat3           38


In [24]:
# 2. Létrehozzuk a listát a napokhoz
df_expanded = df.loc[df.index.repeat(df["napok_szama"])].copy()
# 3. Nap indexet adunk hozzá
df_expanded["nap_a_kezdestol"] = df_expanded.groupby("sorszam").cumcount()
# 4. Dátumot számolunk
df_expanded["datum"] = df_expanded["kezdes_datum"] + pd.to_timedelta(df_expanded["nap_a_kezdestol"], unit="D")

print(df_expanded.head())


   sorszam kezdes_datum  veg_datum kategoria  napok_szama  nap_a_kezdestol  \
0        1   2025-06-11 2025-07-08      kat3           28                0   
0        1   2025-06-11 2025-07-08      kat3           28                1   
0        1   2025-06-11 2025-07-08      kat3           28                2   
0        1   2025-06-11 2025-07-08      kat3           28                3   
0        1   2025-06-11 2025-07-08      kat3           28                4   

       datum  
0 2025-06-11  
0 2025-06-12  
0 2025-06-13  
0 2025-06-14  
0 2025-06-15  


In [25]:
# Kategorizálás a 'nap_a_kezdestol' alapján
df_expanded["kategoriak"] = pd.cut(
    df_expanded["nap_a_kezdestol"],
    bins=[-1, 15, 30, df_expanded["nap_a_kezdestol"].max()],
    labels=["0-15", "16-30", "30+"]
)

# Csoportosítás dátum és kategória szerint
eredmeny = (
    df_expanded
    .groupby(["datum", "kategoriak"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)

# Másolás a vágólapra
eredmeny.reset_index()

C:\Users\laszl\AppData\Local\Temp\ipykernel_5264\2196733741.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["datum", "kategoriak"])


kategoriak,datum,0-15,16-30,30+
0,2025-01-01,280,0,0
1,2025-01-02,588,0,0
2,2025-01-03,836,0,0
3,2025-01-04,1092,0,0
4,2025-01-05,1344,0,0
...,...,...,...,...
409,2026-02-14,0,0,79
410,2026-02-15,0,0,51
411,2026-02-16,0,0,32
412,2026-02-17,0,0,15


In [26]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# --- KONFIGURÁCIÓS BEÁLLÍTÁSOK ---
# Meghatározzuk az idősávok sorrendjét és a hozzájuk tartozó színeket
COLS = ["0-15", "16-30", "30+"]
COLORS = ["green", "orange", "red"]
# A teljes képkeret mérete (szélesség, magasság)
GRAFIKON_MERET = (14, 8)  
# A felhasználói felület (widget) szélessége
WIDGET_SZELESSEG = '400px'

# --- INTERAKTÍV VÁLASZTÓ LISTA (WIDGET) LÉTREHOZÁSA ---
kat_selector = widgets.SelectMultiple(
    options=sorted(df_expanded["kategoria"].unique()), # Kilistázzuk az összes elérhető kategóriát (kat1, kat2...)
    value=tuple(df_expanded["kategoria"].unique()),    # Alapértelmezetten minden ki van jelölve
    description='Kategóriák:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width=WIDGET_SZELESSEG, height='100px') # Megjelenési méret beállítása
)

# Ez a konténer tartja majd a grafikont, hogy frissítéskor ne villogjon/duplázódjon az oldal
out = widgets.Output()

# --- A RAJZOLÓ ÉS ADATFELDOLGOZÓ FÜGGVÉNY ---
def draw(_=None):
    with out:
        # Töröljük az előző grafikont, mielőtt az újat kirajzoljuk
        out.clear_output(wait=True)

        # 1. SZŰRÉS: Csak azokat a sorokat tartjuk meg, amiket a felhasználó kijelölt a listában
        filtered_df = df_expanded[df_expanded["kategoria"].isin(kat_selector.value)]
        
        # 2. CSOPORTOSÍTÁS: Előkészítjük az adatokat a stackplot-hoz
        temp_eredmeny = (
            filtered_df
            .groupby(["datum", "kategoriak"])   # Dátum és idősáv szerint csoportosítunk
            .size()                             # Megszámoljuk a darabszámokat
            .unstack(fill_value=0)              # Az idősávokból oszlopokat csinálunk
            .reindex(columns=COLS, fill_value=0) # Biztosítjuk, hogy minden oszlop meglegyen (akkor is, ha üres)
            .sort_index()                        # Időrendbe rakjuk
        )

        # Ha a szűrés eredménye üres, megállunk
        if temp_eredmeny.empty:
            print("Nincs megjeleníthető adat a választott szűréssel.")
            return

        temp_eredmeny.reset_index()
        
        # 4. NORMALIZÁLÁS: Kiszámoljuk a százalékos megoszlást (soronkénti összeggel osztunk)
        temp_percent = temp_eredmeny.div(temp_eredmeny.sum(axis=1), axis=0) * 100

        # 5. VIZUALIZÁCIÓ: Létrehozunk két egymás alatti grafikont
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=GRAFIKON_MERET)

        # --- Felső ábra: Abszolút darabszám (Stackplot) ---
        ax1.stackplot(temp_eredmeny.index, [temp_eredmeny[col] for col in COLS], 
                      labels=COLS, colors=COLORS, alpha=0.8)
        ax1.set_title(f"Napi darabszám - Szűrve: {', '.join(kat_selector.value)}", fontsize=14)
        ax1.set_ylabel("Darabszám")
        ax1.legend(loc="upper left")
        ax1.grid(True, linestyle="--", alpha=0.5)

        # --- Alsó ábra: Százalékos megoszlás (Stackplot) ---
        ax2.stackplot(temp_percent.index, [temp_percent[col] for col in COLS], 
                      labels=COLS, colors=COLORS, alpha=0.8)
        ax2.set_title("Napi megoszlás (%)", fontsize=14)
        ax2.set_ylabel("Százalék (%)")
        ax2.set_ylim(0, 100) # Fixáljuk a skálát 0 és 100 közé
        ax2.legend(loc="upper left")
        ax2.grid(True, linestyle="--", alpha=0.5)

        # Közös beállítások
        plt.xlabel("Dátum")
        plt.xticks(rotation=45) # Dátumok elforgatása a jobb olvashatóságért
        
        # Automatikus elrendezés finomítása (ne lógjanak egymásba az elemek)
        plt.tight_layout(pad=3.0) 
        plt.show()
        
        # Visszajelzés a frissítés idejéről
        most = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f"Utolsó frissítés: {most}")

# --- ESEMÉNYKEZELÉS ---
# Ha a választólistában változik valami (kijelölés), hívja meg a draw függvényt
kat_selector.observe(draw, names="value")

# --- MEGJELENÍTÉS ---
# Elrendezzük a kezelőfelületet (szűrő felül, grafikon alatta)
display(widgets.VBox([
    widgets.HTML("<b>Kategória szűrő (Több kijelölés: Ctrl / Cmd + Klikk):</b>"),
    kat_selector
]), out)

# Első indítás, hogy ne legyen üres a kép az elején
draw()

Output()